### Python module/package imports for this chapter

In [1]:
import os, sys, collections, re, json, io, base64

In [2]:
import numpy as np

import matplotlib
import matplotlib.pyplot as pp
import matplotlib.animation as anim
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [3]:
import requests
import bs4      # BeautifulSoup 4

In [4]:
import IPython.display
import PIL, PIL.Image, PIL.ImageOps, PIL.ImageEnhance  # Python Imaging Library - now "pillow"

### Stereograph-making Python module, with code taken from previous videos

In [5]:
%%file stereo.py

import os, sys, re, io, base64
import requests, bs4
import PIL, PIL.Image, PIL.ImageOps, PIL.ImageEnhance

def getday(day):
    """Load the webpage collecting Curiosity Front Hazard Cam
    images for Sol day, and yield a sequence of URLs for (left,right) pairs.
    Before loading, see if the webpage is available in a local cache."""
    
    cached = os.path.join('..','03_05','images',str(day) + '.html')
    try:
        text = open(cached,'r').read()
    except FileNotFoundError:
        daypage = requests.get('http://mars.nasa.gov/msl/multimedia/raw',
                               params={'s': day,'camera': 'FHAZ'})
        text = daypage.text
    
    soup = bs4.BeautifulSoup(text,'lxml')
    srcs = [img['src'] for img in soup.find_all('img') if 'Image' in img['alt']]
    
    # drop the smaller thumbnail duplicates
    srcs = srcs[:int(len(srcs)/2)]

    # modify URLs to high-resolution images
    srcs = [re.sub('-thm','',src) for src in srcs]
    
    print("Found {} images for day {}...".format(len(srcs),day))

    # iterate over nonoverlapping pairs in the list:
    # 0,2,4,... and 1,3,5,...
    for one, two in zip(srcs[::2],srcs[1::2]):
        # we may get the left/right in the wrong order, so check the URLs
        left, right = (one, two) if 'FLB' in one else (two, one)
        
    yield left, right

def getimage(url):
    """Load a Curiosity image from url, resize it and dewarp it.
    However, first see if the image is available in a local cache."""
    
    # big = re.sub('-thm','',url)

    cached = os.path.join('..','03_05','images',os.path.basename(url))
    try:
        content = open(cached,'rb').read()
    except FileNotFoundError:
        content = requests.get(url).content
    
    img = PIL.Image.open(io.BytesIO(content))
    
    resized = img.resize((400,400))
    dewarped = img.transform((400,300),
                             PIL.Image.QUAD,data=(0,0,100,400,300,400,400,0),
                             resample=0,fill=1)
    
    return dewarped

def blend(left,right):
    """Colorize and blend left and right Curiosity images."""
    
    blend = PIL.Image.blend(PIL.ImageOps.colorize(left,(0,0,0),(255,0,0)),
                            PIL.ImageOps.colorize(right,(0,0,0),(0,255,255)),0.5)
    
    enhanced = PIL.ImageEnhance.Brightness(blend)
    
    return enhanced.enhance(1.75)

Overwriting stereo.py


## Making a webserver offering stereoscopic Mars images

In [16]:
%%file server.py

import flask

app = flask.Flask(__name__)

@app.route('/')
def hello_world():
    return flask.Response('<html><body><p>Hello, world!</p></body></html>')

app.run(host='0.0.0.0')

Overwriting server.py


In [7]:
%%file server.py

import flask, jinja2, io, base64

app = flask.Flask(__name__)

template = jinja2.Template("""
<html>
  <body>
    <img src="data:image/png;base64,{{imgdata}}"/>
  </body>
</html>
""")

def makeimgdata(img):
    buffer = io.BytesIO()
    img.save(buffer,format='PNG')
    return base64.b64encode(buffer.getvalue()).decode('ascii')

@app.route('/<sol>')
def getsol(sol):
    html = template(imgdata='')
    return flask.Response(html)

@app.route('/')
def hello_world():
    return flask.Response('<html><body><p>Hello, world!</p></body></html>')

app.run(host='0.0.0.0')

Overwriting server.py


In [8]:
img = PIL.Image.open('../03_05/images/FLB_517332079EDR_F0541610FHAZ00323M_.jpg')

In [9]:
def makeimgdata(img):
    buffer = io.BytesIO()
    img.save(buffer,format='PNG')
    return base64.b64encode(buffer.getvalue()).decode('ascii')

In [10]:
b64 = makeimgdata(img)

In [11]:
len(b64)

730876

In [12]:
b64[:1000]

'iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAAAAABadnRfAAEAAElEQVR4nLz9XZckW3IcipmZ+47IrOo+Z74AECBAEOQVpaX7olf9/1+iJ90liSRm5nRXZcR2Nz3syOo+Zwa8AC9ws9fqqoqMjIyP7V/m5u789wCMX7y4fvi7v83nG8TPf1v7mYRNNgiuTc+3yednCNDXJwjw4ye+bXz+Z9i2YcCG175Erz3YIqzQjthSoUGBpHAec57FohQRSpIgSRsEbNcsSDmSjHVck5CVApXueR5ntduG7W/3QSKwTgjwOh3rOh/CkGCQAEitK6UkKYK5b/uvfvv//F/XfTQBVKD1cc8rANOWacJsPm8LzbWf+fx0C4BBGJyJVkMtmA1dR0KrZX78cW3B+hUo8Xk8wNb6ShN4ntW3Deb1/K9TWF+l587ruLi2f7cDKnAdwAQMtvDc0uuM19nB+O6bYeJ5J56vFtYtMdf1WN+99+3WfHxnC9e3tSqum24a346JmWtjxXrnOvd1jwyurQ35QwSuva4faK2r8nU3P3acCYMVLQAVa/Vd3/3tyPp4wBXXDcXHPfruZT7XwfOBtM48NWse88vv/1//v9//5LN88K3f3uBpd5VtN7qNrrVm7b4kUYhx23iSfyr+a5c/2U4DS8A/dvk4vUsBXDtdm/50Ry7x5891y8cx1rfy49v83M3PD/DSBOvukyFBESQBgrDPWbYhMCiJzSWo68TsdhlKUbSBJq6PgiKo6qqyLfj6/o9zImnzqREuBcDvdOHSgd9fFnHpH0DB+O3/4x8+iQpQ7u3Yi88DWTo2z9t7ZrEBqh1FMyc1R5vUOWZg7nj0p3lGnPeTk9tJQz3m/nWfQvjkdjqLAqegGXCDYD62dvaxF7z5dERVGDGlImufQGWpzbqd6L0fGToo91al7EOji4i2YyYMNQjaMmyxhNrPGWwals/Xr8x4jIKlgrMs0yUQKDIml9Za0mNAFaVSqdk5BTuqRxn

In [17]:
%%file server.py

import flask, jinja2, io, base64
import stereo

app = flask.Flask(__name__)

template = jinja2.Template("""
<html>
  <body>
    <img src="data:image/png;base64,{{imgdata}}"/>
  </body>
</html>
""")

def makeimgdata(img):
    buffer = io.BytesIO()
    img.save(buffer,format='PNG')
    return base64.b64encode(buffer.getvalue()).decode('ascii')

@app.route('/<sol>')
def getsol(sol):
    left, right = stereo.getday(sol).__next__()
    img = stereo.blend(stereo.getimage(left),stereo.getimage(right))
    
    html = template.render(imgdata=makeimgdata(img))
    
    return flask.Response(html)

@app.route('/')
def hello_world():
    return flask.Response('<html><body><p>Hello, world!</p></body></html>')

app.run(host='0.0.0.0')

Overwriting server.py
